In [ ]:
#                                                                            
#    Copyright 2022
#    Alexander Belyi <alexander.belyi@gmail.com>,
#    Stanislav Sobolevsky <sobolevsky@nyu.edu>                                               
#                                                                            
#    This file contains the source code of the GNNS algorithm and its evaluation.
#
#    This program is free software: you can redistribute it and/or modify
#    it under the terms of the GNU General Public License as published by
#    the Free Software Foundation, either version 3 of the License, or
#    (at your option) any later version.
#
#    This program is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#    GNU General Public License for more details.
#
#    You should have received a copy of the GNU General Public License
#    along with this program.  If not, see <https://www.gnu.org/licenses/>.
#

In [1]:
!git clone https://github.com/Alexander-Belyi/GNNS.git

Cloning into 'GNNS'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 5), reused 26 (delta 4), pack-reused 0
Unpacking objects: 100% (30/30), done.


In [4]:
!pip install pycombo
!pip install leidenalg
!pip install cdlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 136 kB 31.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 213 kB 58.1 MB/s 
  Using cached importlib_metadata-1.7.0-py2.py3-none-any.whl (31 kB)
  Created wheel for pycombo: filename=pycombo-0.1.7-cp37-cp37m-manylinux_2_27_x86_64.whl size=98830 sha256=9e19beca46226f3c9742a00b868b377976ed462204d0dc244d4b1c47ed62f386
  Stored in directory: /root/.cache/pip/wheels/58/52/18/4c1b80cd45c091e2c1ea442729343ac984dc66b3a678e2c251
Successfully built pycombo
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.4
    Uninstalling importlib-metadata-4.11.4:
      Successfully uninstalled importlib-metadata-4.11.4
ERROR: pip's dependency resolver does not currently take into account all the pac

In [5]:
import bz2
import gc
import time
import torch
import random
import numpy as np
import scipy
import networkx as nx
import igraph as iGraph
import leidenalg
from pycombo import pyCombo
from cdlib import algorithms, NodeClustering
from multiprocessing import Pool
from sklearn.metrics.cluster import normalized_mutual_info_score

Note: to be able to use all crisp methods, you need to install some additional packages:  {'karateclub', 'graph_tool', 'wurlitzer', 'infomap'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'karateclub', 'ASLPAw'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [6]:
# create a tensor
x = torch.tensor([1.0,2.0,3.0,4.0])
print("Tensor:", x)
# check tensor device (cpu/cuda)
print("Tensor device:", x.device)
# Move tensor from CPU to GPU
# check CUDA GPU is available or not
print("CUDA GPU:", torch.cuda.is_available())
if torch.cuda.is_available():
   x=x.to("cuda")
print(x)
# now check the tensor device
print("Tensor device:", x.device)

Tensor: tensor([1., 2., 3., 4.])
Tensor device: cpu
CUDA GPU: True
tensor([1., 2., 3., 4.], device='cuda:0')
Tensor device: cuda:0


In [7]:

class Engine:
    def __init__(self, engine: str) -> None:
        if engine == 'np':
            self.array = self.array_np
            self.sparse_array = self.sparse_array_np
            self.diag = self.diag_np
            self.sum = self.sum_np
            self.sparse_sum = self.sparse_sum_np
            self.mean = self.mean_np
            self.max = self.max_np
            self.argmax = self.argmax_np
            self.eye = self.eye_np
            self.ones = self.ones_np
            self.zeros = self.zeros_np
            self.abs = self.abs_np
            self.exp = self.exp_np
            self.concatenate = self.concatenate_np
            self.reshape = self.reshape_np
            self.transpose = self.transpose_np
            self.tile = self.tile_np
            self.matmul = self.matmul_np
            self.random_uniform = self.random_uniform_np
            self.to_sparse_csr = self.to_sparse_csr_np
            self.cuda = self.identity_np
            self.numpy = self.identity_np
        elif engine == 'torch':
            self.device = torch.device('cpu')
            self.cuda_available = False
            if torch.cuda.is_available():
                self.cuda_available = True
                self.device = torch.device(0)
            self.array = self.array_torch
            self.sparse_array = self.sparse_array_torch
            self.diag = self.diag_torch
            self.sum = self.sum_torch
            self.sparse_sum = self.sparse_sum_torch
            self.mean = self.mean_torch
            self.max = self.max_torch
            self.argmax = self.argmax_torch
            self.eye = self.eye_torch
            self.ones = self.ones_torch
            self.zeros = self.zeros_torch
            self.abs = self.abs_torch
            self.exp = self.exp_torch
            self.concatenate = self.concatenate_torch
            self.reshape = self.reshape_torch
            self.transpose = self.transpose_torch
            self.tile = self.tile_torch
            self.matmul = self.matmul_torch
            self.random_uniform = self.random_uniform_torch
            self.to_sparse_csr = self.to_sparse_csr_torch
            self.cuda = self.cuda_torch
            self.numpy = self.numpy_torch

    def array_np(self, x, device=None):
        return np.array(x)

    def sparse_array_np(self, x, device=None):
        #if type(x) == scipy.sparse.csr_array: # requires python >= 3.8
        if type(x) == scipy.sparse.csr_matrix:
            return x
        #return scipy.sparse.csr_array(x)
        return scipy.sparse.csr_matrix(x)

    def diag_np(self, x):
        return np.diag(x)

    def sum_np(self, x, axis=None, keepdims=False):
        return np.sum(x, axis=axis, keepdims=keepdims)

    def sparse_sum_np(self, x, axis=None):
        return np.array(np.sum(x, axis=axis))

    def mean_np(self, x, axis=None, keepdims=False):
        return np.mean(x, axis=axis, keepdims=keepdims)
    
    def max_np(self, x, axis=None, keepdims=False):
        return np.max(x, axis=axis, keepdims=keepdims)

    def argmax_np(self, x, axis=None): 
        return np.argmax(x, axis=axis)

    def eye_np(self, size, dtype=None, device=None):
        return np.eye(size, dtype=dtype)

    def ones_np(self, size, dtype=None, device=None):
        return np.ones(size, dtype=dtype)

    def zeros_np(self, size, dtype=None, device=None):
        return np.zeros(size, dtype=dtype)

    def abs_np(self, x):
        return np.abs(x)

    def exp_np(self, x):
        return np.exp(x)

    def concatenate_np(self, x, axis=None):
        return np.concatenate(x, axis=axis)
    
    def reshape_np(self, x, shape):
        return np.reshape(x, shape)

    def transpose_np(self, x, axis1, axis2):
        return np.swapaxes(x, axis1, axis2)

    def tile_np(self, x, shape):
        return np.tile(x, shape)

    def matmul_np(self, x, y):
        return np.matmul(x, y)

    def random_uniform_np(self, low, high, size, device="cpu"):
        return np.random.uniform(low=low, high=high, size=size)

    def to_sparse_csr_np(self, x):
        return x.tocsr()

    def identity_np(self, x):
        return x

    ### torch
    def array_torch(self, x, device="cpu"):
        if not self.cuda_available:
            device = "cpu"
        return torch.tensor(x, device=device)

    def sparse_array_torch(self, x, device="cpu"):
        if not self.cuda_available:
            device = "cpu"
        #coo_x = scipy.sparse.coo_array(x) # requires python >=3.8
        coo_x = scipy.sparse.coo_matrix(x)
        return torch.sparse_coo_tensor(np.vstack((coo_x.row, coo_x.col)), coo_x.data, size=coo_x.shape, device=device)

    def diag_torch(self, x):
        return torch.diag(x)

    def sum_torch(self, x, axis=None, keepdims=False):
        return torch.sum(x, axis=axis, keepdims=keepdims)

    def sparse_sum_torch(self, x, axis=None):
        return torch.sparse.sum(x, dim=axis).to_dense()

    def mean_torch(self, x, axis=None, keepdims=False):
        return torch.mean(x, axis=axis, keepdims=keepdims)
    
    def max_torch(self, x, axis=None, keepdims=False):
        if axis is None:
            return torch.max(x, axis=axis, keepdims=keepdims)
        else:
            return torch.max(x, axis=axis, keepdims=keepdims).values

    def argmax_torch(self, x, axis=None, device="cpu"):
        if not self.cuda_available:
            device = "cpu"
        return torch.argmax(x, axis=axis).to(device)

    def eye_torch(self, size, dtype=None, device="cpu"):
        if not self.cuda_available:
            device = "cpu"
        return torch.eye(size, dtype=dtype, device=device)

    def ones_torch(self, size, dtype=None, device="cpu"):
        if not self.cuda_available:
            device = "cpu"
        return torch.ones(size, dtype=dtype, device=device)

    def zeros_torch(self, size, dtype=None, device="cpu"):
        if not self.cuda_available:
            device = "cpu"
        return torch.zeros(size, dtype=dtype, device=device)

    def abs_torch(self, x):
        return torch.abs(x)

    def exp_torch(self, x):
        return torch.exp(x)

    def concatenate_torch(self, x, axis=0):
        return torch.cat(x, dim=axis)
    
    def reshape_torch(self, x, shape):
        return torch.reshape(x, shape)

    def transpose_torch(self, x, dim0, dim1):
        return torch.transpose(x, dim0, dim1)

    def tile_torch(self, x, shape):
        return torch.tile(x, shape)

    def matmul_torch(self, x, y):
        return torch.matmul(x, y)

    def random_uniform_torch(self, low, high, size, dtype=None, device="cpu"):
        if not self.cuda_available:
            device = "cpu"
        return torch.zeros(size, dtype=dtype, device=device).uniform_(low, high)

    def to_sparse_csr_torch(self, x):
        return x.to_sparse_csr()

    def cuda_torch(self, x):
        if not self.cuda_available:
            return x
        return x.cuda()
    
    def numpy_torch(self, x):
        return x.cpu().numpy()


In [8]:
def set_all_random_seeds(seed=1):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)


def get_modularity_matrix(G, symmetrize=True, loops_style=2, device="cpu"):
    '''build modularity matrix'''
    A = nx.to_numpy_array(G)
    A = eng.array(A, device)
    if loops_style == 2 and not G.is_directed():
        A += eng.diag(eng.diag(A))
    elif loops_style == 0:
        A -= eng.diag(eng.diag(A))
    w_in = A.sum(axis=0, keepdims=True)
    w_out = A.sum(axis=1, keepdims=True)
    T = w_out.sum()
    Q = A / T - w_out @ w_in / (T ** 2)
    if symmetrize:
        Q = (Q + Q.T) / 2
    return Q

def get_sparse_modularity_matrix(G, loops_style=2, device="cpu"):
    '''build sparse modularity matrix'''
    #A = nx.to_scipy_sparse_array(G, format="coo") #requires python >= 3.8
    A = nx.to_scipy_sparse_matrix(G, dtype=float, format="coo")
    if loops_style == 2 and not G.is_directed():
        A.setdiag(A.diagonal() * 2)
    elif loops_style == 0:
        A.setdiag(0)
    Q_diag = eng.array(A.diagonal(), device)
    A = eng.sparse_array(A, device)
    w_in = eng.sparse_sum(A, axis=0).reshape((1, A.shape[0]))
    w_out = eng.sparse_sum(A, axis=1).reshape((A.shape[0], 1))
    T = w_out.sum()
    w_in /= T
    w_out /= T
    Q = A / T
    Q_diag /= T
    Q = eng.to_sparse_csr(Q)
    # Q -= w_out @ w_in
    return Q, Q_diag, w_out, w_in


In [9]:
class GNNSModularityOptimizer():
    def __init__(self, G, strip_diagonal=True, normalize_modularity=False,
                 normalize_each_step=True, normalize_QC=True, use_sparse=False):
        self.num_model_params = 2
        self.net_size = len(G)
        self.normalize_modularity = normalize_modularity
        self.use_sparse = use_sparse
        #strip diagonal elements of the modularity matrix when initializing the data
        self.strip_diagonal = strip_diagonal
        if self.use_sparse:
            self.sparse_Q, self.Q_diag, self.w_out, self.w_in = get_sparse_modularity_matrix(G, device="cuda:0")
            self.Q_diag = eng.reshape(self.Q_diag, (1, self.net_size, 1))
            self.Q_diag -= eng.reshape(self.w_out * eng.transpose(self.w_in, 0, 1), (1, self.net_size, 1))
        else:
            self.Q = get_modularity_matrix(G, symmetrize=True, device="cuda:0")
            self.Q_diag = eng.reshape(eng.diag(self.Q), (1, self.net_size, 1))
        self.reshape_Q()
        self.normalize_each_step = normalize_each_step
        self.normalize_QC = normalize_QC

    def reshape_Q(self):
        if self.use_sparse:
            self.w_out = self.w_out[None, :, :]
            self.w_in = self.w_in[None, :, :]
        else:
            self.Q = self.Q.reshape((1, *self.Q.shape)) # add batch dimension
            if self.normalize_modularity:
                w = eng.sum(eng.abs(self.Q), axis=2, keepdims=True)
                self.Q /= w + (w==0)

    def reshape_model_params(self, params):
        f0 = eng.cuda(-params[:, 0:1, None])
        f1 = eng.cuda(params[:, 1:2, None])
        f2 = 1.0 - f0 - f1
        return f0, f1, f2

    def discretize(self):
        c = eng.argmax(self.C, axis=2)
        self.C[:,:,:] = 0
        for i in range(self.batch_size):
            self.C[i, range(self.net_size), c[i, :]] = 1

    def calculate_modularity(self):
        QxC = self.Q_times_C(False)
        q = eng.matmul(self.C.reshape((self.batch_size, 1, -1)), QxC.reshape((self.batch_size, -1, 1))).reshape((-1,))
        return q

    def activation(self,x): #ReLU
        return x * (x>0)

    def Q_times_C(self, strip_diagonal):
        if self.use_sparse:
            # Stack the vector batch into columns. (b, n, k) -> (n, b, k) -> (n, b*k)
            C = eng.transpose(self.C, 0, 1).reshape((self.net_size, -1))
            # And then reverse the reshaping. (n, n) x (n, b*k) = (n, b*k) -> (n, b, k) -> (b, n, k)
            QxC = eng.transpose((self.sparse_Q @ C).reshape((self.net_size, self.batch_size, -1)), 1, 0)
            QxC -= eng.matmul(self.w_out, eng.matmul(self.w_in, self.C))
        else:
            QxC = eng.matmul(self.Q, self.C)
        if strip_diagonal:
            return QxC - self.Q_diag * self.C
        else:
            return QxC

    def calculate(self, params, init_partition, num_iterations, discretize_result=False):
        self.batch_size = init_partition.shape[0] # number of random starting configurations
        f0, f1, f2 = self.reshape_model_params(params)
        # C.shape = batch_size x net_size x num_communities
        self.C = init_partition
        self.normalize_attachments()
        for _ in range(num_iterations):
            QxC = self.Q_times_C(self.strip_diagonal)
            if self.normalize_QC: #normalize by max QxC
                t = eng.abs(eng.max(QxC, axis=2, keepdims=True))
                QxC /= t + (t == 0)
            bias = eng.ones((self.batch_size, self.net_size, 1), dtype=float, device="cuda:0")
            next_C = f0 * bias + f1 * self.C + f2 * QxC
            self.C = self.activation(next_C)
            self.normalize_attachments()
        if discretize_result:
            self.discretize()
        mod = self.calculate_modularity()
        return self.C, mod

    def normalize_attachments(self):
        if self.normalize_each_step:
            w = eng.sum(self.C, axis=2, keepdims=True)
            self.C /= w + (w==0)


In [10]:

def runGNNSSeries(G, max_num_communities, iterations_per_stage, num_random_configs, fraction_to_keep,
                discretize_last=True, init_params=None, init_communities=None, alpha=0.0, manual_gc=True, verbose=0):
    num_model_params = 2
    net_size = len(G)
    max_batch_size = hypers.get('max_batch_size', 1000)
    max_total_tensor_size = hypers.get('max_total_tensor_size', 200_000_000)
    max_batch_size = max(1, min(max_batch_size, int(max_total_tensor_size / (net_size * max_num_communities))))
    #print('net_size', net_size,
    #      'max_num_communities =', max_num_communities,
    #      'num_random_configs =', num_random_configs,
    #      'max_batch_size =', max_batch_size,
    #      'net_size * max_num_communities =',
    #      net_size * max_num_communities)
    start_time = time.time()
    GNNS = GNNSModularityOptimizer(G, strip_diagonal=hypers.get('strip_diagonal', True),
                                    normalize_modularity=hypers.get('normalize_modularity', False),
                                    normalize_each_step=hypers.get('normalize_each_step', True),
                                    use_sparse=hypers.get('use_sparse', False))
    final_modularities = np.empty(0)
    best_modularity = -1
    # if num_random_configs is too large, do calculations in chunks
    for i in range((num_random_configs + max_batch_size - 1) // max_batch_size):
        cur_range = (i * max_batch_size, min((i + 1) * max_batch_size, num_random_configs))
        batch_size = cur_range[1] - cur_range[0]
        if init_params is None:
            params = eng.random_uniform(low=0, high=1.0, size=(batch_size, num_model_params), device="cuda:0")
        else:
            params = eng.tile(init_params.flatten(), (batch_size, 1))
        if init_communities is None:
            partition = eng.random_uniform(low=0, high=1.0, size=(batch_size, len(G), max_num_communities), device="cuda:0")
        else:
            partition = eng.tile(init_communities, (batch_size, 1, 1))
        for stage in range(len(iterations_per_stage)):
            discretize = discretize_last and (stage == len(iterations_per_stage) - 1)
            communities, modularities = GNNS.calculate(params, partition, iterations_per_stage[stage], discretize_result=discretize)
            modularities = eng.numpy(modularities)
            index_best = np.argmax(modularities)
            if verbose > 0:
                print('Stage {} completed'.format(stage+1))
                print('Top modularity={}, mean={}, best_parameters={}'.format(modularities[index_best], np.mean(modularities), params[index_best]))
            if stage < len(iterations_per_stage) - 1:
                next_batch_size = max(1, batch_size * iterations_per_stage[0] // iterations_per_stage[stage+1])
                selected_indices = modularities >= sorted(modularities)[-max(1, int(next_batch_size * fraction_to_keep))]
                params = params[selected_indices, :]
                # initialize with best partitions while keeping initial model params
                partition = communities[selected_indices]
                num_to_keep = sum(selected_indices)
                num_to_repeat = next_batch_size - num_to_keep
                if num_to_repeat > 0:
                    # some king of soft-max; uniform for alpha=0
                    weights = np.exp(modularities[selected_indices] * alpha)
                    weights = weights / sum(weights)
                    indices_to_repeat_partition = np.random.choice(num_to_keep, size=num_to_repeat, p=weights)
                    repeated_partition = partition[indices_to_repeat_partition]
                    partition = eng.concatenate([partition, repeated_partition], axis=0) # we copy initial partitions with best modularities
                    # old way:
                    #indices_to_repeat_params = np.random.choice(num_to_keep, size=num_to_repeat, p=weights)
                    #repeated_params = params[indices_to_repeat_params, :num_model_params] # randomly permute repeated params
                    #params = eng.concatenate([params, repeated_params], axis=0)
                    new_params = eng.random_uniform(low=0, high=1.0, size=(num_to_repeat, num_model_params), device="cuda:0")
                    params = eng.concatenate([params, new_params], axis=0) # replace repeated params with random ones
        final_modularities = np.concatenate([final_modularities, modularities])
        if modularities[index_best] > best_modularity:
            best_modularity = modularities[index_best]
            best_communities = communities[index_best, :]
            best_parameters = params[index_best]
        if manual_gc:
            del params, partition
            gc.collect()
            torch.cuda.empty_cache()
    if manual_gc:
        del GNNS
        gc.collect()
        torch.cuda.empty_cache()
    return best_communities, modularities, best_parameters, best_modularity, time.time()-start_time


In [11]:

def apply_method(arg):
    G, weight_attr, method, seed = arg
    modularity = -1
    start_time = time.time()
    if method == 'combo':
        partition, modularity = pyCombo.execute(G, random_seed=seed)
        partition_sets = {comm: set() for comm in partition.values()}
        for node, comm in partition.items():
            partition_sets[comm].add(node)
        cdlib_comms = NodeClustering([list(c) for c in partition_sets.values()], G, "Combo", method_parameters={})
    elif method == 'leiden':
        partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights=weight_attr, n_iterations=-1, seed=seed)
        cdlib_comms = NodeClustering([G.vs[x]["_nx_name"] for x in partition], G, "Leiden", method_parameters={})
    elif method == 'leiden_ig':
        partition = G.community_leiden(weights=weight_attr, objective_function="modularity", n_iterations=-1)
        cdlib_comms = NodeClustering([G.vs[x]["_nx_name"] for x in partition], G, "Leiden_ig", method_parameters={})
    elif method == 'louvain':
        cdlib_comms = algorithms.louvain(G)
    elif method == 'louvain_ig':
        partition = G.community_multilevel(weights=weight_attr)
        cdlib_comms = NodeClustering([G.vs[x]["_nx_name"] for x in partition], G, "Luvain_ig", method_parameters={})
        modularity = partition.modularity
    elif method == 'belief':
        cdlib_comms = algorithms.belief(G)
    elif method ==  'eigenvector':
        partition = G.community_leading_eigenvector()
        cdlib_comms = NodeClustering([G.vs[x]["_nx_name"] for x in partition], G, "Eigenvector", method_parameters={})
        modularity = partition.modularity
    elif method ==  'greedy_modularity':
        cdlib_comms = algorithms.greedy_modularity(G)
    elif method ==  'greedy_modularity_ig':
        partition = G.community_fastgreedy().as_clustering()
        cdlib_comms = NodeClustering([G.vs[x]["_nx_name"] for x in partition], G, "CNM", method_parameters={})
        modularity = partition.modularity
    elif method ==  'spinglass':
        try:
            partition = G.community_spinglass()
            cdlib_comms = NodeClustering([G.vs[x]["_nx_name"] for x in partition], G, "Spinglass", method_parameters={})
            modularity = partition.modularity
        except:
            cdlib_comms = NodeClustering([[0] * len(G.vs)], G, "Spinglass", method_parameters={})
            modularity = 0
    return cdlib_comms, modularity, time.time() - start_time

def is_igraph_method(method):
    return method == 'leiden' or method == 'leiden_ig' or method == 'louvain_ig' or method == 'eigenvector' or method == 'spinglass' or method == 'greedy_modularity_ig'

def is_deterministic(method):
    return method == 'greedy_modularity' or method == 'greedy_modularity_ig'


In [12]:

def partitionSeries(G, method, num_runs=10, verbose=0):
    if num_runs > 0:
        if is_deterministic(method):
            num_runs = 1
        partition = None
        best_mod = -1
        total_time = 0
        results = eng.zeros((num_runs, 2)) # (mod, num_comms) pairs
        graph = G
        weight_attr = None
        if is_igraph_method(method):
            graph = iGraph.Graph.from_networkx(G)
            if nx.is_weighted(G):
                weight_attr = 'weight'
        for i in range(num_runs):
            cdlib_comms, modularity, elapsed_time = apply_method((graph, weight_attr, method, hypers['seed']+i))
            if is_igraph_method(method):
                cdlib_comms.graph = G
            total_time += elapsed_time
            if modularity == -1:
                results[i, 0] = cdlib_comms.newman_girvan_modularity().score
            else:
                results[i, 0] = modularity
            results[i, 1] = len(cdlib_comms.communities)
            if best_mod < results[i, 0]:
                best_mod = results[i, 0]
                partition = cdlib_comms
            if verbose > 1:
                print(method + "   mod_cdlib:", results[i, 0])
        max_mod_index = eng.argmax(results[:, 0])
        if verbose > 0:
            print('Best ' + method + ' modularity={}; {} communities'.format(results[max_mod_index, 0], results[max_mod_index, 1]))
        res = {'best': results[max_mod_index, 0],
               'best1': results[0, 0],
               'best5': max(results[:5, 0]),
               'best10': max(results[:10, 0]),
               'best20': max(results[:20, 0]),
               'min': results[:, 0].min(),
               'mean': results[:, 0].mean(),
               'comm_number': int(results[max_mod_index, 1]),
               'partition': partition,
               'total_time': total_time,
               'avg_time': total_time / num_runs}
    else:
        res = {}
    return res

def partitionSeries_parallel(G, method, num_runs=10, verbose=0):
    if num_runs > 0:
        if is_deterministic(method):
            num_runs = 1
        partition = None
        best_mod = -1
        total_time = 0
        results = eng.zeros((num_runs, 2)) # (mod, num_comms) pairs
        graph = G
        weight_attr = None
        if is_igraph_method(method):
            graph = iGraph.Graph.from_networkx(G)
            if nx.is_weighted(G):
                weight_attr = 'weight'
        with Pool(hypers["num_processes"]) as pool:
            imap_unordered_it = pool.imap_unordered(apply_method,
                                                    [(graph, weight_attr, method, hypers['seed']+i) for i in range(num_runs)],
                                                    chunksize=hypers["num_processes"])
            i = 0
            for res in imap_unordered_it:
                cdlib_comms, modularity, elapsed_time = res
                if is_igraph_method(method):
                    cdlib_comms.graph = G
                total_time += elapsed_time
                if modularity == -1:
                    results[i, 0] = cdlib_comms.newman_girvan_modularity().score
                else:
                    results[i, 0] = modularity
                results[i, 1] = len(cdlib_comms.communities)
                if best_mod < results[i, 0]:
                    best_mod = results[i, 0]
                    partition = cdlib_comms
                if verbose > 1:
                    print(method + "   mod_cdlib:", results[i, 0])
                i += 1
        max_mod_index = eng.argmax(results[:, 0])
        if verbose > 0:
            print('Best ' + method + ' modularity={}; {} communities'.format(results[max_mod_index, 0], results[max_mod_index, 1]))
        res = {'best': results[max_mod_index, 0],
               'best1': results[0, 0],
               'best5': max(results[:5, 0]),
               'best10': max(results[:10, 0]),
               'best20': max(results[:20, 0]),
               'min': results[:, 0].min(),
               'mean': results[:, 0].mean(),
               'comm_number': int(results[max_mod_index, 1]),
               'partition': partition,
               'total_time': total_time,
               'avg_time': total_time / num_runs}
    else:
        res = {}
    return res


In [19]:

def processNet(G, methods, num_runs=20, nums_initial_GNNS_configs=(100), verbose=0):
    if num_runs > 0:
        results = {}
        comm_number = 0
        best_mod = -1
        for method in methods:
            #results[method] = partitionSeries_parallel(G, num_runs=num_runs, method=method, verbose=verbose - 1)
            results[method] = partitionSeries(G, num_runs=num_runs, method=method, verbose=verbose - 1)
            if best_mod < results[method].get('best', 0):
                best_mod = results[method].get('best', 0)
                comm_number = max(comm_number, results[method].get('comm_number', 0))
            if verbose > 0:
                print(method + ' modularity best/best1/best5/best10/best20 = {:.6f}/{:.6f}/{:.6f}/{:.6f}/{:.6f}; min/avg = {:.6f}/{:.6f}; {} comm; time total/avg {:.2f}/{:.2f}'.format(
                                    results[method].get('best', 0),
                                    results[method].get('best1', 0),
                                    results[method].get('best5', 0),
                                    results[method].get('best10', 0),
                                    results[method].get('best20', 0),
                                    results[method].get('min', 0),
                                    results[method].get('mean', 0),
                                    results[method].get('comm_number', 0),
                                    results[method].get('total_time', 0),
                                    results[method].get('avg_time', 0)))
        if comm_number <= 0:
            comm_number = 50
        comm_number = min(comm_number, 50)
        GNN = {}
        for num_initial_GNNS_configs in nums_initial_GNNS_configs:
            set_all_random_seeds(hypers['seed'])
            iterations_per_stage = [10, 10, 30]
            if num_initial_GNNS_configs >= 1_000:
                iterations_per_stage += [100]
            if num_initial_GNNS_configs >= 10_000:
                iterations_per_stage += [350]
            fraction_to_keep = 1/3 #proportion of best configs to move to the next stage
            C, _, best_params, best_mod, GNNS_time = runGNNSSeries(G, max_num_communities = comm_number + 1,
                                                    iterations_per_stage=iterations_per_stage,
                                                    num_random_configs=num_initial_GNNS_configs,
                                                    fraction_to_keep=fraction_to_keep,
                                                    manual_gc=(len(G) > 1000),
                                                    verbose=verbose-1)
            GNN[num_initial_GNNS_configs] = {'mod':best_mod,
                                             'best_params':best_params,
                                             'partition': C.argmax(axis=1),
                                             'total_time': GNNS_time}
            if verbose > 0:
                print('GNN{} modularity = {}; best params = {}'.format(num_initial_GNNS_configs, GNN[num_initial_GNNS_configs]['mod'], GNN[num_initial_GNNS_configs]['best_params']))
        #latex table output
        print(' & '.join([G.name,
                        ' & '.join(['%.6f' % results[method].get('best',0) for method in methods]),
                        ' & '.join(['%.6f' % GNN[v].get('mod',0) for v in nums_initial_GNNS_configs]),
                        ' & '.join(['%.2f' % results[method].get('total_time',0) for method in methods]),
                        ' & '.join(['%.2f' % GNN[v].get('total_time',0) for v in nums_initial_GNNS_configs])
                        ]))
        return results, GNN


In [15]:

def read_graph(name, make_undirected=True, remove_weights=False, verbose=0):
    if name[-4:] == ".net":
        G = nx.read_pajek(hypers['path_classic'] + name)
        G.name = name[:-4]
    elif name[-10:] == ".graph.bz2":
        with bz2.open(hypers['path_classic'] + name) as f:
            G = nx.DiGraph()
            i = 0
            n = 0
            m = 0
            fmt = 0
            for line in f:
                line = line.strip()
                if len(line) == 0:
                    if i > 0:
                        i += 1
                elif line[0] != '%':
                    values = [int(x) for x in line.split()]
                    if i == 0:
                        n, m = values[:2]
                        if len(values) > 2:
                            fmt = values[2]
                        if fmt > 1:
                            print("Error in file: ", name, " unsupported format!")
                            break
                        G.add_nodes_from(range(n))
                    else:
                        if fmt == 0:
                            G.add_edges_from([(i-1, j-1) for j in values])
                        elif fmt == 1:
                            G.add_weighted_edges_from([(i-1, j-1, w) for j, w in zip(values[::2], values[1::2])])
                    i += 1
        G.name = name[:-10]
    num_selfloops = len(list(nx.selfloop_edges(G)))
    if num_selfloops > 0:
        print(name, " self-loops: ", num_selfloops)
    A = nx.to_scipy_sparse_matrix(G)
    is_weighted = len(np.unique(A.data)) > 2 #there are values other than 0 and 1
    is_directed = np.abs((A - A.transpose()).data).sum() > 1e-10
    if remove_weights:
        G = nx.from_scipy_sparse_matrix(A > 0)
    if make_undirected or not is_directed:
        G = nx.Graph(G)
    if verbose > 0:
        print('{} of size {}, with {} edges, directed = {}, weighted = {}, #self-loops = {}'.format(
            name, len(G), G.number_of_edges(), is_directed, is_weighted, num_selfloops))
    del A
    gc.collect()
    return G


In [16]:
torch.set_default_dtype(torch.float64)

hypers = {}
hypers['path_classic'] = './GNNS/networks/'

#hypers['ENGINE'] = 'np'
hypers['ENGINE'] = 'torch'
eng = Engine(hypers['ENGINE'])

hypers['seed'] = 13
hypers['strip_diagonal'] = True
hypers['normalize_modularity'] = False
hypers['normalize_each_step'] = True
hypers['use_sparse'] = True
hypers['max_batch_size'] = 1000
hypers['max_total_tensor_size'] = 100_000_000
hypers["num_processes"] = 4


In [22]:

def get_real_world_nets_small():
    network_file_names = [
        "karate.graph.bz2",
        "chesapeake.graph.bz2",
        "dolphins.graph.bz2",
        "lesmis.graph.bz2",
        "polbooks.graph.bz2", 
        "adjnoun.graph.bz2",
        "football.graph.bz2",
        "jazz.graph.bz2",
        "celegansneural.graph.bz2",
        "USAir97_undir.net",
        "celegans_metabolic.graph.bz2",
        "email.graph.bz2",
        "polblogs.graph.bz2",
        "US Airports Network_undir.net",
    ]
    return network_file_names

def get_real_world_nets_big():
    network_file_names = [
        "power.graph.bz2",
        "PGPgiantcompo.graph.bz2",
        "krong500slogn16.graph.bz2",
    ]
    return network_file_names

def run_real_world_nets(big_nets=False):
    network_file_names = get_real_world_nets_small()
    if big_nets:
        network_file_names = get_real_world_nets_big()
    methods = ['leiden', 'louvain_ig']#, 'spinglass', 'greedy_modularity_ig']#, 'eigenvector', 'belief']
    if not big_nets:
        methods += ['combo']
    nums_initial_GNNS_configs = [100, 2500]
    #latex table output
    print(' & '.join(['Network', ' & '.join([method+'_mod' for method in methods]),
                    ' & '.join(['GNN'+str(v)+'_mod' for v in nums_initial_GNNS_configs]),
                    ' & '.join([method+'_time' for method in methods]),
                    ' & '.join(['GNN'+str(v)+'_time' for v in nums_initial_GNNS_configs])]))
    verbose = 0
    for name in network_file_names:
        if verbose > 0:
            print("\n\n\n" + name)
        G = read_graph(name, make_undirected=True)
        processNet(G, methods, nums_initial_GNNS_configs=nums_initial_GNNS_configs, num_runs=10, verbose=verbose)
        gc.collect()
        torch.cuda.empty_cache()
        

run_real_world_nets()

Network & leiden_mod & louvain_ig_mod & combo_mod & GNN100_mod & GNN2500_mod & leiden_time & louvain_ig_time & combo_time & GNN100_time & GNN2500_time
karate & 0.419790 & 0.419790 & 0.419790 & 0.419790 & 0.419790 & 0.00 & 0.00 & 0.01 & 0.03 & 0.23
chesapeake & 0.265796 & 0.265796 & 0.265796 & 0.265796 & 0.265796 & 0.01 & 0.00 & 0.01 & 0.03 & 0.22
dolphins & 0.528519 & 0.524109 & 0.526799 & 0.528519 & 0.528519 & 0.01 & 0.00 & 0.01 & 0.03 & 0.24
lesmis & 0.566688 & 0.566688 & 0.566688 & 0.566688 & 0.566688 & 0.01 & 0.00 & 0.03 & 0.03 & 0.26
polbooks & 0.527237 & 0.526967 & 0.527237 & 0.527237 & 0.527237 & 0.04 & 0.02 & 0.29 & 0.11 & 0.48
adjnoun & 0.309570 & 0.303934 & 0.310461 & 0.308758 & 0.310967 & 0.02 & 0.01 & 0.12 & 0.03 & 0.33
football & 0.604570 & 0.604570 & 0.604570 & 0.602872 & 0.604570 & 0.01 & 0.00 & 0.08 & 0.03 & 0.34
jazz & 0.445144 & 0.445144 & 0.444469 & 0.445144 & 0.445144 & 0.03 & 0.02 & 0.11 & 0.03 & 0.33
celegansneural & 0.503485 & 0.498211 & 0.503782 & 0.502002 & 0.5

In [ ]:
gc.collect()
torch.cuda.empty_cache()
run_real_world_nets(True)
gc.collect()
torch.cuda.empty_cache()

Network & leiden_mod & louvain_ig_mod & GNN100_mod & GNN2500_mod & leiden_time & louvain_ig_time & GNN100_time & GNN2500_time
power & 0.940714 & 0.936614 & 0.818490 & 0.880699 & 6.28 & 1.12 & 1.43 & 39.60
PGPgiantcompo & 0.886640 & 0.884784 & 0.838746 & 0.865027 & 11.41 & 2.59 & 3.32 & 101.95
krong500slogn16 & 0.063362 & 0.059964 & 0.058143 & 0.064676 & 5983.00 & 298.22 & 45.78 & 1061.30


In [ ]:

def generate_BM_series(num_nets, block_sizes, inner_prob_mult_factor):
    # inner_prob_mult_factor - how much stronger are the inner edges
    block_sizes = np.array(block_sizes) # sizes of the blocks
    average_prob = 0.1 # average probability of connections
    net_size = sum(block_sizes)
    num_blocks = len(block_sizes)
    Gs = [0] * num_nets
    selfloops = False
    for i in range(num_nets):
        num_inner_edges = (block_sizes * (block_sizes - 1 + selfloops)).sum()
        num_total_edges = net_size * (net_size - 1 + selfloops)
        prob_of_inner_edge = num_inner_edges / num_total_edges
        # average_prob = outer_prob*(1-prob_of_inner_edge) + inner_prob_mult_factor*outer_prob*prob_of_inner_edge =>
        outer_prob = average_prob / ((inner_prob_mult_factor - 1) * prob_of_inner_edge + 1) #probability between groups
        P = outer_prob * (np.ones((num_blocks, num_blocks)) + np.eye(num_blocks) * (inner_prob_mult_factor - 1))
        communities = {i: sum(np.cumsum(block_sizes) <= i) for i in range(net_size)}
        Gs[i] = (nx.stochastic_block_model(block_sizes, P, selfloops=selfloops), communities)
    return Gs

def generate_LFR_series(num_nets=10, net_size=250, mu=0.1, min_degree=20, max_degree=50, min_community=20):
    hypers['LFR_tau1']=2.0
    hypers['LFR_tau2']=1.1
    Gs = [0] * num_nets
    i = 0
    j = 0
    while i < num_nets:
        try:
            G = nx.LFR_benchmark_graph(net_size, tau1=hypers['LFR_tau1'], tau2=hypers['LFR_tau2'], mu=mu,
                                        min_degree=min_degree, max_degree=max_degree, min_community=min_community, seed=hypers['seed']+j)
            G.name = 'LFR' + str(i)
            comms = {}
            for v in G.nodes:
                if v in comms:
                    label = comms[v]
                else:
                    label = len(np.unique(list(comms.values())))
                    for u in G.nodes[v]['community']:
                        comms[u] = label
            Gs[i] = (G, comms)
            i += 1
        except:
            pass
        j += 1
    return Gs

def evalSeries(num_nets=10, net_size=150, netType='BM', num_initial_GNNS_configs=100, num_runs=1,
                bm_inner_prob_mult_factor=1.5, lfr_mu=0.35, lfr_min_degree=20, lfr_max_degree=50, lfr_min_community=20):
    def NMI(part1, part2): #NMI between two partition dictionaries
        n = len(part1)
        p2 = [0] * n
        if type(part2) is list:
            for label, comm in enumerate(part2):
                for v in comm:
                    p2[v] = label
        else:
            for v, comm in enumerate(part2):
                p2[v] = eng.numpy(comm)
        p1 = [part1[p] for p in range(n)]
        return normalized_mutual_info_score(p1, p2)

    seed = hypers['seed']
    set_all_random_seeds(seed)
    if netType == 'BM':
        G = generate_BM_series(num_nets, (net_size//3, net_size//3, net_size//3), bm_inner_prob_mult_factor)
    else:
        G = generate_LFR_series(num_nets, net_size, lfr_mu, lfr_min_degree, lfr_max_degree, lfr_min_community)
        num_nets = len(G)
    methods = ['leiden', 'louvain_ig', 'combo']
    num_methods = len(methods) + 1
    mods = np.zeros((num_nets, num_methods))
    nmi = np.zeros((num_nets, num_methods))
    elapsedTime = [0] * num_methods
    total_avg_degree = 0
    for i in range(num_nets):
        avg_degree = sum(list(zip(*list(G[i][0].degree())))[1]) / len(G[i][0])
        print('Network {}/{}, size = {}, avg. degree = {}'.format(i+1, num_nets, len(G[i][0]), avg_degree))
        res, resGNN = processNet(G[i][0], methods, num_runs, [num_initial_GNNS_configs], verbose=0)
        mods[i, :] = [res[method]['best'] for method in methods] + [resGNN[num_initial_GNNS_configs]['mod']]
        nmi[i,:] = [NMI(G[i][1], res[method]['partition'].communities) for method in methods] + [NMI(G[i][1], resGNN[num_initial_GNNS_configs]['partition'])]
        for j, method in enumerate(methods):
            elapsedTime[j] += res[method]['total_time'] / num_nets
        elapsedTime[3] += resGNN[num_initial_GNNS_configs]['total_time'] / num_nets
        total_avg_degree += avg_degree / num_nets
    print("average degree = ", total_avg_degree)
    print('methods = %s & %s & %s & %s'%tuple(methods + ['GNNS'+str(num_initial_GNNS_configs)]))
    print('Mod = %.6f & %.6f & %.6f & %.6f'%tuple(mods.mean(axis=0)))
    print('NMI = %.6f & %.6f & %.6f & %.6f'%tuple(nmi.mean(axis=0)))
    print('Time = %.3f & %.3f & %.3f & %.3f'%tuple(elapsedTime))


In [ ]:
def run_synthetic_nets():
    evalSeries(num_nets=10, net_size=250, netType = 'LFR', num_initial_GNNS_configs=100, num_runs=20,
            lfr_mu=0.45, lfr_min_degree=15, lfr_max_degree=40, lfr_min_community=20)
    evalSeries(num_nets=10, net_size=250, netType = 'LFR', num_initial_GNNS_configs=100, num_runs=20,
            lfr_mu=0.45, lfr_min_degree=5, lfr_max_degree=30, lfr_min_community=20)
    for factor in (1.5, 2.0, 2.5, 3.0):
        evalSeries(num_nets=10, net_size=300, netType = 'BM', num_initial_GNNS_configs=100, num_runs=20,
                   bm_inner_prob_mult_factor=factor)

run_synthetic_nets()

Network 1/10, size = 250, avg. degree = 29.768
LFR0 & 0.227704 & 0.227139 & 0.227455 & 0.227575 & 0.50 & 0.29 & 5.64 & 0.04
Network 2/10, size = 250, avg. degree = 28.432
LFR1 & 0.233267 & 0.232769 & 0.233812 & 0.233420 & 0.50 & 0.24 & 7.14 & 0.05
Network 3/10, size = 250, avg. degree = 30.2
LFR2 & 0.221316 & 0.220588 & 0.221756 & 0.220485 & 0.61 & 0.28 & 5.76 & 0.04
Network 4/10, size = 250, avg. degree = 29.984
LFR3 & 0.221645 & 0.218156 & 0.221137 & 0.221001 & 0.60 & 0.26 & 6.42 & 0.04
Network 5/10, size = 250, avg. degree = 30.28
LFR4 & 0.216868 & 0.215753 & 0.217309 & 0.217130 & 0.58 & 0.34 & 9.06 & 0.05
Network 6/10, size = 250, avg. degree = 29.536
LFR5 & 0.238262 & 0.237164 & 0.238262 & 0.238180 & 0.43 & 0.25 & 5.74 & 0.05
Network 7/10, size = 250, avg. degree = 29.344
LFR6 & 0.243997 & 0.243267 & 0.244261 & 0.243424 & 0.43 & 0.23 & 5.75 & 0.05
Network 8/10, size = 250, avg. degree = 29.376
LFR7 & 0.242806 & 0.242207 & 0.242889 & 0.242216 & 0.52 & 0.24 & 5.10 & 0.04
Network 9/1